In [1]:
import azureml.core

print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.")

You are currently using version 1.38.0 of the Azure ML SDK.


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "NC6Cluster"

try:
    compute_target = ws.compute_targets[cluster_name]
    print("Found existing compute target.")
except KeyError:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_NC6",
        idle_seconds_before_scaledown=600,
        min_nodes=0,
        max_nodes=4,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(
    show_output=True, min_node_count=None, timeout_in_minutes=20
)

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.core import Experiment

experiment_name = "automl-image-multiclass-xray"
experiment = Experiment(ws, name=experiment_name)

In [5]:
from azureml.core import Dataset
from azureml.data import DataType

# get existing training dataset
training_dataset_name = "XrayTrainingDataset"
if training_dataset_name in ws.datasets:
    training_dataset = ws.datasets.get(training_dataset_name)
    print("Found the training dataset", training_dataset_name)

# get existing validation dataset
validation_dataset_name = "XrayValidationDataset"
if validation_dataset_name in ws.datasets:
    validation_dataset = ws.datasets.get(validation_dataset_name)
    print("Found the validation dataset", validation_dataset_name)


Found the training dataset XrayTrainingDataset
Found the validation dataset XrayValidationDataset


In [6]:
training_dataset.to_pandas_dataframe()

,image_url,label
0,workspaceblobstore/Xray/Covid/COVID19(59).jpg,Covid
1,workspaceblobstore/Xray/Covid/COVID19(6).jpg,Covid
2,workspaceblobstore/Xray/Covid/COVID19(60).jpg,Covid
3,workspaceblobstore/Xray/Covid/COVID19(61).jpg,Covid
4,workspaceblobstore/Xray/Covid/COVID19(62).jpg,Covid
...,...,...
130,workspaceblobstore/Xray/Pneumonia/person994_vi...,Pneumonia
131,workspaceblobstore/Xray/Pneumonia/person995_ba...,Pneumonia
132,workspaceblobstore/Xray/Pneumonia/person995_vi...,Pneumonia
133,workspaceblobstore/Xray/Pneumonia/person996_vi...,Pneumonia


In [7]:
from azureml.automl.core.shared.constants import ImageTask
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import GridParameterSampling, choice

image_config_resnet = AutoMLImageConfig(
    task=ImageTask.IMAGE_CLASSIFICATION,
    compute_target=compute_target,
    training_data=training_dataset,
    validation_data=validation_dataset,
    hyperparameter_sampling=GridParameterSampling({"model_name": choice("resnet34")}),
    iterations=1,
)

In [8]:
automl_image_run = experiment.submit(image_config_resnet)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-image-multiclass-xray,AutoML_54c2bfd4-18fa-4e53-9a5a-44df9f9c2d74,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
automl_image_run.wait_for_completion(wait_post_processing=True)

In [12]:
from azureml.automl.core.shared.constants import ImageTask
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import BanditPolicy, RandomParameterSampling
from azureml.train.hyperdrive import choice, uniform

parameter_space = {
    "learning_rate": uniform(0.001, 0.01),
    "optimizer": choice("sgd", "adam"),
    "model": choice(
        {
            "model_name": choice("vitb16r224", "vits16r224"),
            "number_of_epochs": choice(15, 30),
        },
        {
            "model_name": choice("seresnext", "resnet18", "resnest50", "resnest101"),
            "layers_to_freeze": choice(0, 2),
        },
    ),
}

tuning_settings = {
    "iterations": 10,
    "max_concurrent_iterations": 2,
    "hyperparameter_sampling": RandomParameterSampling(parameter_space),
    "early_termination_policy": BanditPolicy(
        evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
    ),
}

automl_image_config = AutoMLImageConfig(
    task=ImageTask.IMAGE_CLASSIFICATION,
    compute_target=compute_target,
    training_data=training_dataset,
    validation_data=validation_dataset,
    experiment_timeout_hours=6,
    **tuning_settings,
)

In [13]:
automl_image_run = experiment.submit(automl_image_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-image-multiclass-xray,AutoML_f7d265c6-dfa9-4105-b55b-b6e9673d1401,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
automl_image_run.wait_for_completion(wait_post_processing=True)

In [ ]:
# Register the model from the best run

best_child_run = automl_image_run.get_best_child()
model_name = best_child_run.properties["model_name"]
model = best_child_run.register_model(
    model_name=model_name, model_path="outputs/model.pt"
)